# Configuration Refactoring Plan for ActualGameSearch Worker

## Problem Statement
The Worker has accumulated many magic strings for configuration keys throughout the codebase. We need to systematically refactor to use strongly-typed options patterns following .NET Core best practices.

## Current State Analysis
Based on our workspace tree and recent grep searches, we have identified:
- Multiple magic strings in Program.cs for configuration access
- Hardcoded values mixed with configuration-driven values 
- No central configuration options classes
- Inconsistent patterns for accessing configuration

## Goal: Idiomatic .NET Core Configuration
- Strongly-typed options classes with validation
- Centralized configuration key constants
- Dependency injection of options
- Clean separation between legacy and new configuration patterns

In [ ]:
# Import Required Libraries
import pandas as pd
import json
import os
import re
from pathlib import Path

# Set up workspace root
workspace_root = Path("/workspaces/ActualGameSearch_V3")
print(f"Working in: {workspace_root}")

# Define the tree output from the user's command
tree_output = """ActualGameSearch_V3/ (/workspaces/ActualGameSearch_V3)
├── .github/
├── .specify/
├── .venv/
├── .vscode/
├── AI-Agent-Workspace/
├── infrastructure/
├── specs/
├── src/
│   ├── ActualGameSearch.Api/
│   ├── ActualGameSearch.AppHost/
│   ├── ActualGameSearch.Core/
│   ├── ActualGameSearch.ServiceDefaults/
│   ├── ActualGameSearch.Worker/
│   │   ├── Configuration/
│   │   │   ├── ConfigurationKeys.cs
│   │   │   ├── ConfigurationExtensions.cs
│   │   │   └── WorkerOptions.cs
│   │   ├── Embeddings/
│   │   ├── Ingestion/
│   │   ├── Models/
│   │   ├── Probes/
│   │   ├── Processing/
│   │   ├── Services/
│   │   ├── Storage/
│   │   └── Program.cs
│   └── AI-Agent-Workspace/
├── tests/
└── [config files]"""

print("Tree output loaded for analysis")

## Configuration Refactoring Steps

### Step 1: Current Magic Strings Analysis
Let's analyze where configuration magic strings are used in the codebase.

In [ ]:
# Step 1: Analyze Magic Strings Usage
magic_strings_found = {
    "Ollama": ["Ollama:Endpoint", "Ollama:Model"],
    "Cosmos": ["Cosmos:Database", "Cosmos:GamesContainer", "Cosmos:ReviewsContainer", 
               "Cosmos:Vector:Path", "Cosmos:Vector:Dimensions", "Cosmos:Vector:DistanceFunction"],
    "DataLake": ["DataLake:Root"],
    "Ingestion": ["Ingestion:MaxConcurrency", "Ingestion:ReviewCapBronze", "Ingestion:NewsCountBronze"],
    "Candidacy": ["Candidacy:Bronze:MinRecommendationsForInclusion", 
                  "Candidacy:Bronze:MinRecommendationsForEmbedding",
                  "Candidacy:Bronze:MaxAssociatedAppIds"],
    "Seeding": ["Seeding:MaxReviewsPerGame", "Seeding:FailOnNoReviews", "Seeding:MinReviewCount"],
    "Embeddings": ["Embeddings:AllowDeterministicFallback", "Embeddings:NumCtx", "Embeddings:MaxBatch"]
}

# Count total magic strings
total_strings = sum(len(strings) for strings in magic_strings_found.values())
print(f"Total magic strings identified: {total_strings}")

for category, strings in magic_strings_found.items():
    print(f"{category}: {len(strings)} keys")
    for string in strings:
        print(f"  - {string}")
    print()

### Step 2: Define New Configuration Structure

We've already created:
- `WorkerOptions.cs` - Strongly-typed options classes with validation
- `ConfigurationKeys.cs` - Constants to eliminate magic strings  
- `ConfigurationExtensions.cs` - DI setup helpers
- `IEmbeddingService.cs` - Refactored embedding service

### Step 3: Refactoring Strategy

**Current Problem**: Program.cs has 900+ lines with mixed concerns:
- Configuration parsing with magic strings
- DI setup mixed with business logic
- Command parsing embedded in startup
- Multiple responsibilities in one file

**Solution**: Progressive refactoring without breaking existing functionality:

1. **Phase 1**: Keep existing structure, replace magic strings with constants
2. **Phase 2**: Inject strongly-typed options into services that need them
3. **Phase 3**: Refactor Program.cs to use dependency injection properly

In [ ]:
# Step 3: Implementation Plan - Phase 1 Quick Wins

# Let's start with safe replacements that won't break existing functionality
phase1_replacements = {
    # Most common magic strings in Program.cs that can be easily replaced
    '"Ollama:Endpoint"': 'ConfigurationKeys.Legacy.Ollama.Endpoint',
    '"Ollama:Model"': 'ConfigurationKeys.Legacy.Ollama.Model',
    '"Cosmos:Database"': 'ConfigurationKeys.Legacy.Cosmos.Database',
    '"Cosmos:GamesContainer"': 'ConfigurationKeys.Legacy.Cosmos.GamesContainer',
    '"Cosmos:ReviewsContainer"': 'ConfigurationKeys.Legacy.Cosmos.ReviewsContainer',
    '"DataLake:Root"': 'ConfigurationKeys.Legacy.DataLake.Root',
}

print("Phase 1: Safe magic string replacements")
for old_string, new_constant in phase1_replacements.items():
    print(f"  {old_string} → {new_constant}")

print("\nBenefits of Phase 1:")
print("- Eliminates magic strings without changing behavior")
print("- Provides central location for all config keys")
print("- Makes refactoring easier in future phases")
print("- No breaking changes to existing command-line interface")

In [ ]:
# Step 4: Services That Need Strongly-Typed Options

services_to_refactor = {
    "BronzeStoreIngestor": {
        "current": "IConfiguration configuration",
        "new": "IOptions<CandidacyOptions> candidacyOptions",
        "benefits": "Type-safe access to candidacy thresholds"
    },
    "EmbeddingService": {
        "current": "Static utility methods",
        "new": "IEmbeddingService with IOptions<EmbeddingOptions>", 
        "benefits": "Proper DI, testability, configuration validation"
    },
    "SteamHttpClient": {
        "current": "Direct configuration access",
        "new": "IOptions<SteamOptions>",
        "benefits": "Centralized Steam API configuration"
    }
}

print("Services prioritized for strongly-typed options:")
for service, details in services_to_refactor.items():
    print(f"\n{service}:")
    print(f"  Current: {details['current']}")
    print(f"  New: {details['new']}")  
    print(f"  Benefits: {details['benefits']}")

print("\n" + "="*50)
print("RECOMMENDATION: Start with Phase 1")
print("Replace magic strings with constants first.")
print("This provides immediate value with minimal risk.")